In [1]:
import sys

import nivapy3 as nivapy
import pandas as pd

sys.path.append("..")

import useful_rid_code as rid

# Extracting water chemistry and discharge data for Øyvind K/Liv Bente for 1990 to 2020

See e-mail from Liv Bente received 08.09.2022 at 11.39 for details.

In [2]:
# Stations of interest
stn_ids = [
    29617,
    29612,
    29615,
    29613,
    29614,
    29783,
    29778,
    29782,
    29779,
]

# Pars of interest
par_list = [
    "SPM",
    "TOC",
    "PO4-P",
    "TOTP",
    "NO3-N",
    "NH4-N",
    "TOTN",
    "Si",
    "SiO2",
    "pH",
    "KOND",
    "TURB860",
]

# Period of interest
st_year, end_year = 1990, 2020

In [3]:
eng = nivapy.da.connect()

Username:  ···
Password:  ···············


Connection successful.


In [4]:
in_xlsx = r"../../../../Data/RID_Sites_List_2017-2020.xlsx"
stn_df = pd.read_excel(in_xlsx, sheet_name="RID_20")
stn_df = stn_df.query("station_id in @stn_ids")
stn_df

,station_id,station_code,station_name,old_rid_group,new_rid_group,ospar_region,station_type,nve_vassdrag_nr,lat,lon,utm_north,utm_east,utm_zone
1,29612,BUSEDRA,Drammenselva,rid_11,rid_20,SKAGERAK,R,012.A3,59.753995,10.008990,6624446.00,556695.00,32
2,29779,FINEALT,Altaelva,rid_11,rid_20,LOFOTEN-BARENTS SEA,R,212.A0,69.900992,23.286977,7759686.00,586586.00,34
7,29782,NOREVEF,Vefsna,rid_11,rid_20,NORWEGIAN SEA2,R,151.A4,65.749000,13.239000,7293064.00,419297.00,33
10,29783,ROGEORR,Orreelva,rid_11,rid_20,NORTH SEA,R,028.4A,58.731000,5.529000,6512861.00,646406.00,31
14,29778,STREORK,Orkla,rid_11,rid_20,NORWEGIAN SEA2,R,121.A41,63.200996,9.772994,7008450.27,538750.37,32
15,29613,TELESKI,Skienselva,rid_11,rid_20,SKAGERAK,R,016.A221,59.199000,9.611000,6562920.00,534741.00,32
17,29614,VAGEOTR,Otra,rid_11,rid_20,SKAGERAK,R,021.A11,58.187000,7.954000,6450006.00,438495.00,32
18,29615,VESENUM,Numedalslågen,rid_11,rid_20,SKAGERAK,R,015.A1,59.086000,10.070000,6551828.42,561347.37,32
19,29617,ØSTEGLO,Glomma ved Sarpsfoss,rid_11,rid_20,SKAGERAK,R,002.A51,59.279802,11.134111,6573156.00,621600.00,32


In [5]:
wc_df_list = []
q_df_list = []

for idx, row in stn_df.iterrows():
    stn_id = row["station_id"]
    stn_code = row["station_code"]

    # Get WC data
    wc_df, dup_df = rid.extract_water_chem(
        stn_id,
        par_list,
        "%s-01-01" % st_year,
        "%s-12-31" % end_year,
        eng,
        plot=False,
        samp_sel=63,
    )
    cols = sorted(wc_df.columns)
    wc_df.reset_index(inplace=True)
    wc_df["station_id"] = stn_id
    wc_df["station_code"] = stn_code

    # Reorder columns
    cols = ["station_id", "station_code", "sample_date"] + cols
    wc_df = wc_df[cols]
    del wc_df["pH_flag"], wc_df["TURB860_flag"]

    wc_df_list.append(wc_df)

    # Get flow data
    q_df = rid.extract_discharge(
        stn_id, "%s-01-01" % st_year, "%s-12-31" % end_year, eng, plot=False
    )
    q_df.columns = [f"{stn_code}_q_m3ps"]

    q_df_list.append(q_df)

wc_df = pd.concat(wc_df_list, axis="rows")
q_df = pd.concat(q_df_list, axis="columns").reset_index()

    WARNING
    The database contains duplicated values for some station-date-parameter combinations.
    Only the most recent values will be used, but you should check the repeated values are not errors.
    The duplicated entries are returned in a separate dataframe.

    WARNING
    The database contains duplicated values for some station-date-parameter combinations.
    Only the most recent values will be used, but you should check the repeated values are not errors.
    The duplicated entries are returned in a separate dataframe.

    WARNING
    The database contains duplicated values for some station-date-parameter combinations.
    Only the most recent values will be used, but you should check the repeated values are not errors.
    The duplicated entries are returned in a separate dataframe.

    WARNING
    The database contains duplicated values for some station-date-parameter combinations.
    Only the most recent values will be used, but you should check the repeated value

In [6]:
# Save
chem_xlsx = r"../../../../Results/Loads_CSVs/rid_water_chem_oka.xlsx"
wc_df.to_excel(chem_xlsx, index=False)

q_xlsx = r"../../../../Results/Loads_CSVs/rid_flow_oka.xlsx"
q_df.to_excel(q_xlsx, index=False)